### setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np
import datetime
import random

In [3]:
import pandas as pd

# import gspread
# from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [6]:
# scope = ["https://spreadsheets.google.com/feeds"]
# credentials = ServiceAccountCredentials.from_json_keyfile_name(
#    "jupyter-integration-credentials.json", scope
# )
# gc = gspread.authorize(credentials)

In [7]:
# spreadsheet_key = "1_RXzXkHPEyDAiDKmz98wTSC9UNo-8OyjGTT540X_vJk"
# book = gc.open_by_key(spreadsheet_key)

In [8]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [9]:
# Opening the worksheet by using Worksheet ID
# workbook = gc.open_by_key(spreadsheet_key)
# Selecting which sheet to pulling the data
# sheet = workbook.worksheet("Sheet1")
# Pulling the data and transform it to the data frame
# values = sheet.get_all_values()

### read in data

#### by race

In [10]:
df = pd.read_excel(
    "data/raw/vsrr023-tables2.xlsx", sheet_name="Table 44", skiprows=2, skipfooter=1
)

In [11]:
df = df[df.columns.drop(list(df.filter(regex="Unnamed")))]

In [12]:
df.columns = [
    "year",
    "overall",
    "male",
    "female",
    "hispanic",
    "hispanic_male",
    "hispanic_female",
    "aian",
    "aian_male",
    "aian_female",
    "aapi",
    "aapi_male",
    "aapi_female",
    "black",
    "black_male",
    "black_female",
    "white",
    "white_male",
    "white_female",
]

In [13]:
df = df.drop(index=6).reset_index()

In [14]:
df = df[~df["overall"].isna()].drop(columns="index")

In [15]:
df["year"] = df["year"].str[:4]

In [16]:
years = ["2019", "2020", "2021"]

In [17]:
df_race = pd.melt(
    df[df["year"].isin(years)],
    id_vars=["year"],
    value_vars=[
        "overall",
        "male",
        "female",
        "hispanic",
        "hispanic_male",
        "hispanic_female",
        "aian",
        "aian_male",
        "aian_female",
        "aapi",
        "aapi_male",
        "aapi_female",
        "black",
        "black_male",
        "black_female",
        "white",
        "white_male",
        "white_female",
    ],
)

#### sketching out race

In [18]:
df_race[~df_race["variable"].str.contains("male")]

,year,variable,value
0,2021,overall,76.1
1,2020,overall,77
2,2019,overall,78.8
9,2021,hispanic,77.7
10,2020,hispanic,77.9
11,2019,hispanic,81.9
18,2021,aian,65.2
19,2020,aian,67.1
20,2019,aian,71.8
27,2021,aapi,83.5


In [19]:
race_pct_change = (
    df[df.columns.drop(list(df.filter(regex="male")))]
    .replace("---", np.nan)
    .sort_values("year", ascending=True)
)
race_pct_change["overall"] = (
    race_pct_change["overall"].astype(float)
    - (race_pct_change["overall"].astype(float)).shift(periods=1)
) / (race_pct_change["overall"].astype(float)).shift(periods=1)

race_pct_change["hispanic"] = (
    race_pct_change["hispanic"].astype(float)
    - (race_pct_change["hispanic"].astype(float)).shift(periods=1)
) / (race_pct_change["hispanic"].astype(float)).shift(periods=1)

race_pct_change["aian"] = (
    race_pct_change["aian"].astype(float)
    - (race_pct_change["aian"].astype(float)).shift(periods=1)
) / (race_pct_change["aian"].astype(float)).shift(periods=1)

race_pct_change["aapi"] = (
    race_pct_change["aapi"].astype(float)
    - (race_pct_change["aapi"].astype(float)).shift(periods=1)
) / (race_pct_change["aapi"].astype(float)).shift(periods=1)


race_pct_change["black"] = (
    race_pct_change["black"].astype(float)
    - (race_pct_change["black"].astype(float)).shift(periods=1)
) / (race_pct_change["black"].astype(float)).shift(periods=1)

race_pct_change["white"] = (
    race_pct_change["white"].astype(float)
    - (race_pct_change["white"].astype(float)).shift(periods=1)
) / (race_pct_change["white"].astype(float)).shift(periods=1)

In [20]:
race_pct_change = pd.melt(
    race_pct_change,
    id_vars="year",
    value_vars=["overall", "hispanic", "aian", "aapi", "black", "white"],
)

In [21]:
alt.Chart(race_pct_change).mark_bar().encode(
    x=alt.X("year"),
    y=alt.Y("value"),
    color=alt.condition(
        alt.datum.value > 0, alt.value("#00eeef"), alt.value("#D95F1A")
    ),
).properties(width=300, height=120).facet(facet=alt.Facet("variable:O"), columns=2)

# color=alt.condition(
#        alt.datum.nonfarm_change > 0,
#        alt.value("steelblue"),  # The positive color
#        alt.value("orange")

alt.FacetChart(...)

In [22]:
alt.Chart(race_pct_change).mark_bar().encode(
    x=alt.X("year"), y=alt.Y("value")
).properties(width=150, height=150).facet(
    data=race_pct_change, facet=alt.Facet("variable", columns=4)
)

SchemaValidationError: Invalid specification

        altair.vegalite.v4.api.FacetChart->facet, validating 'anyOf'

        {'columns': 4, 'field': 'variable', 'type': 'nominal'} is not valid under any of the given schemas
        

alt.FacetChart(...)

#### sketching out overall trends

In [23]:
df_overall = df.drop(list(df)[2:19], axis=1)

In [24]:
alt.Chart(df_overall).mark_bar().encode(x="year", y="overall").properties(width=500)

alt.Chart(...)

In [25]:
df_overall = df_overall.sort_values("year", ascending=True)

In [26]:
df_overall["pct_change"] = (
    df_overall["overall"] - df_overall["overall"].shift(periods=1)
) / df_overall["overall"].shift(periods=1)

In [27]:
df_overall["yearly_chg"] = df_overall["overall"] - df_overall["overall"].shift(
    periods=1
)

In [28]:
alt.Chart(df_overall).mark_bar().encode(x="year", y="pct_change").properties(width=300)

alt.Chart(...)

In [29]:
alt.Chart(df_overall).mark_bar().encode(x="year", y=alt.Y("yearly_chg")).properties(
    width=300
)

alt.Chart(...)

In [30]:
dw.add_data(chart_id="cC8q7", data=df_overall)

<Response [204]>

#### making and adding df for range chart

In [31]:
dw_race = df[df.columns.drop(list(df.filter(regex="male")))].replace("---", "")

In [32]:
dw_race = dw_race[dw_race["year"].isin(years)]

In [33]:
dw.create_chart(chart_type="d3-arrow-plot", data=dw_race)

New chart d3-arrow-plot created!


{'publicId': 'RKnoW',
 'language': 'en-US',
 'theme': 'grid',
 'title': 'New Chart',
 'type': 'd3-arrow-plot',
 'metadata': {'data': {'changes': [],
   'transpose': False,
   'vertical-header': True,
   'horizontal-header': True},
  'describe': {'source-name': '',
   'source-url': '',
   'intro': '',
   'byline': '',
   'aria-description': '',
   'number-format': '-',
   'number-divisor': 0,
   'number-append': '',
   'number-prepend': ''},
  'visualize': {'dark-mode-invert': True,
   'highlighted-series': [],
   'highlighted-values': [],
   'sharing': {'enabled': False},
   'y-grid-labels': 'outside'},
  'axes': {},
  'publish': {'embed-width': 360,
   'embed-height': 400,
   'blocks': {'logo': {'enabled': True, 'id': 'main'},
    'embed': False,
    'download-pdf': False,
    'download-svg': False,
    'get-the-data': False,
    'download-image': False},
   'export-pdf': {}},
  'annotate': {'notes': ''},
  'custom': {}},
 'authorId': 393404,
 'id': 'RKnoW',
 'lastModifiedAt': '2022-0